In [1]:
import pandas as pd
from sklearn import linear_model
from regressors import stats
from sklearn import svm
from sklearn.ensemble import ExtraTreesRegressor
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [2]:
#leemos el df resultado del proceso anterior y borramos los NA
df_or = pd.read_csv("input.csv")
df = pd.read_csv("input.csv")
df_or.dropna(inplace = True)
df.dropna(inplace = True)

In [3]:
df["Total_edad"] = df["20-24"]+df["25-59"]+df["5-19"]+df["60+"]
df["Total_sexo"] = df["HOMBRE"]+df["MUJER"]
df["Total_educacion"] = df["MEDIA"]+df["NINGUNA"]+df["POSGRADO"]+df["SUPERIOR"]
df["20-24"] = df["20-24"]/df["Total_edad"]
df["25-59"] = df["25-59"]/df["Total_edad"]
df["5-19"] = df["5-19"]/df["Total_edad"]
df["60+"] = df["60+"]/df["Total_edad"]
df["MEDIA"] = df["MEDIA"]/df["Total_educacion"]
df["NINGUNA"] = df["NINGUNA"]/df["Total_educacion"]
df["POSGRADO"] = df["POSGRADO"]/df["Total_educacion"]
df["SUPERIOR"] = df["SUPERIOR"]/df["Total_educacion"]
df["HOMBRE"] = df["HOMBRE"]/df["Total_sexo"]
df["MUJER"] = df["MUJER"]/df["Total_sexo"]
df.drop(["Total_sexo", "Total_educacion", "Total_edad"], axis = 1, inplace = True)
df.dropna(inplace = True)

In [4]:
#preparamos los datos para el modelo, en especial la columna de USO
le = preprocessing.LabelEncoder()
scaler = StandardScaler()

In [5]:
#reemplazamos en la misma columna de USO el label encoder de scikit
le.fit(df["USO"])
df["USO"] = le.transform(df["USO"])

In [6]:
#estandarizacion de datos y transformacion del df original listo para el modelo
df = pd.DataFrame(df, columns = df_or.columns)
df = df.drop(["ZAT"], axis = 1)

In [7]:
#definimos las variables x, y de nuestro modelo
x = df.drop(["N_ORIGEN", "N_DESTINO"], axis = 1)
y = df["N_ORIGEN"]
y2 = df["N_DESTINO"]

In [8]:
#regresion ridge
reg_ridge = linear_model.Ridge(alpha=0)
reg_ridge.fit(x, y)
reg_ridge.coef_
reg_ridge.intercept_
reg_ridge.score(x, y)

0.08568236623879522

In [9]:
#arbol de decision
tree = ExtraTreesRegressor(n_estimators=50)
tree.fit(x, y2)
tree.feature_importances_

array([0.06167751, 0.01405335, 0.0267939 , 0.0361858 , 0.06815984,
       0.08685367, 0.09411143, 0.12494941, 0.0726129 , 0.1288619 ,
       0.06202204, 0.0766404 , 0.07272212, 0.07435573])

In [10]:
#al pasarle el resultado nos dice cuales son las variables mas importantes con respecto a al mean
imp = tree.feature_importances_
mean = np.mean(imp)
bool_array = imp >= mean
indices = []
indices2 = []
H = 0
H2 = 0
for q in range(len(bool_array)):
    if ~bool_array[q]:
        indices.append(H)
    H = H+1     
for w in range(len(bool_array)):
    if bool_array[w]:
        indices2.append(H2)
    H2 = H2+1      
columnas_eliminar = []
columnas_restantes = []
for ind in indices:
    columnas_eliminar.append(x.columns[ind])
for ind in indices2:
    columnas_restantes.append(x.columns[ind])

In [11]:
#variables que no son importantes para el modelo
columnas_eliminar

['ESTRATO', 'USO', 'CUPOS', 'PARQUEADEROS', '20-24', 'POSGRADO']

In [12]:
#variables significativas para el modelo
columnas_restantes

['25-59', '5-19', '60+', 'MEDIA', 'NINGUNA', 'SUPERIOR', 'HOMBRE', 'MUJER']

In [13]:
df = df[columnas_restantes]

In [14]:
#regresion ridge
reg_ridge = linear_model.Ridge(alpha=0)
reg_ridge.fit(df, y)
reg_ridge.coef_
reg_ridge.intercept_
reg_ridge.score(df, y)

C:\Users\DNCIESQUENAZI\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=3.42245e-19): result may not be accurate.
  overwrite_a=True).T


0.06390735565687

In [15]:
reg_ridge.coef_

array([ -2389.44204838,  -7970.81304296,  -8795.00193401,   1743.10040511,
       -14799.60534589,   1454.24760821,  18670.1483886 ,  27301.07574684])